# Import Statements

In [1]:
import pandas as pd
import numpy as np
import string
import os
import re
import pickle

In [2]:
from sortedcontainers import SortedDict, SortedList, SortedSet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

# Read From Files

In [3]:
def getListOfFiles(directory):
    '''
    Parameters:
        directory: type(string)
        
    returns: list of all files in directory with the full path of file
    '''
    
    list_of_files = []
    
    for file_path in os.listdir(directory):
        full_path = os.path.join(directory, file_path)
        if os.path.isfile(full_path):
            list_of_files.append(full_path)
    
    return list_of_files

# Preprocessing Functions

In [4]:
def lowercase(data):
    '''
    Parameters:
        data: type(string)
    
    returns: lowercase of data
    '''
    
    return data.lower()

In [5]:
def perform_word_tokenize(corpus):
    '''
    Parameters:
        corpus: type(string)
    
    returns word-level tokenization of corpus
    '''
    
    return word_tokenize(corpus)

In [6]:
def remove_stopwords_from_tokens(tokens, stopwords_set):
    '''
    Parameters:
        tokens: type(list)
        stopwords_set: type(set)
    
    returns: tokens without stopwords
    '''
    tokens_sans_stopwords = [x for x in tokens if x not in stopwords_set]
    
    return tokens_sans_stopwords

In [7]:
def remove_punctuation_from_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without punctuation
    '''
    tokens_sans_punctuation = [x.translate(str.maketrans('', '', string.punctuation)) for x in tokens]
    
    return tokens_sans_punctuation

In [8]:
def remove_blank_space_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: tokens without blank tokens
    '''
    tokens_sans_blank_space = [x for x in tokens if x!='']
    
    return tokens_sans_blank_space

In [9]:
def lemmatize_tokens(tokens):
    '''
    Parameters:
        tokens: type(list)
    
    returns: returns unique tokens after lemmatization
    '''
    lemmatizer = WordNetLemmatizer()
    lemmatize_tokens = [lemmatizer.lemmatize(x) for x in tokens]
    unique_lemmatize_tokens = list(dict.fromkeys(lemmatize_tokens))
    
    return unique_lemmatize_tokens

In [10]:
def preprocess(corpus, stopwords_set, preprocess_type):
    # Convert the text to lower case
    lowercase_corpus = lowercase(corpus)
    #print(len(lowercase_corpus))
    
    # Perform word tokenization (word_tokenize also takes care of whitespace)
    word_tokens = perform_word_tokenize(lowercase_corpus)
    #print(len(word_tokens))
    
    # Remove stopwords from tokens
    word_tokens_sans_stopwords = remove_stopwords_from_tokens(word_tokens, stopwords_set)
    #print(len(word_tokens_sans_stopwords))
    
    # Remove punctuation marks from tokens
    word_tokens_sans_punctuation = remove_punctuation_from_tokens(word_tokens_sans_stopwords)
    #print(len(word_tokens_sans_punctuation))
    
    # Remove blank space tokens
    word_tokens_sans_blank_tokens = remove_blank_space_tokens(word_tokens_sans_punctuation)
    #print(len(word_tokens_sans_blank_tokens))
    
    # Lemmatize tokens
    #word_tokens_final = lemmatize_tokens(word_tokens_sans_blank_tokens)
    #print(len(word_tokens_final))
    
    return word_tokens_sans_blank_tokens

# Helper Functions

In [11]:
def create_file_dictionary(list_of_files):
    '''
    Paramteres:
        list_of_files: type(string)
    
    returns: file_dictionary with integer key and path_of_file as value
    '''
    file_dictionary = {}
    for i in range(len(list_of_files)):
        file_dictionary[i] = list_of_files[i]
    
    return file_dictionary

In [12]:
def create_positional_index(file_dictionary, stopwords_set):
    # initialize positional index
    positional_index = {}
    
    # positional index
    for doc_ID in range(len(file_dictionary)):
        file = open(file_dictionary[doc_ID], 'r', encoding='utf-8', errors='ignore')
        file_corpus = file.read()
        file.close()
        doc_tokens = preprocess(file_corpus, stopwords_set, 'doc')
        for j, word in enumerate(doc_tokens):
            if(word in positional_index):
                if(doc_ID in positional_index[word]):
                    positional_index[word][doc_ID].append(j)
                else:
                    positional_index[word][doc_ID] = [j]
            else:
                positional_index[word] = {doc_ID:[j]}
#         for token in doc_tokens:
#             if token in positional_index:
#                 positional_index[token][0] += 1
#                 if doc_ID in positional_index[token][1]:
#                     positional_index[token][1][doc_ID] = doc_tokens.index(token)
#                 else:
#                     positional_index[token][1] = SortedDict({doc_ID:positional_index[token]})
#             else:
#                 positional_index[token] = [1, SortedDict()]
#                 positional_index[token][1] = SortedDict({doc_ID:positional_index[token]})
    
    # Storing positional index
    pi_file = open('positional_index_pickle_file', 'wb')
    pickle.dump(positional_index, pi_file)
    pi_file.close()

# Main

In [13]:
def get_common_keys(old_common_keys, new_list):
    new_common_keys = [x for x in old_common_keys if x in new_list]
    return new_common_keys

In [14]:
# def combine_positional_indexes(list1, list2):
#     common_keys = [x for x in list1 if x in list2]
#     common_dict = {}
#     for ck in common_keys:
#         if ck==27 or ck==209:
#             common_dict[ck] = ['what','to','do']
#     return common_keys

In [27]:
def main():
    # create set of stop words for preprocessing
    stopwords_set = set(stopwords.words('english'))
    
    # Get List of Files in Dataset
    list_of_files = getListOfFiles('Dataset/Humor,Hist,Media,Food/')
    
    # create dictionary of file with docID (integer) as key and full_path of file as value
    file_dictionary = create_file_dictionary(list_of_files)
    
    # create positional index once and then load pickle file afterwards
    #create_positional_index(file_dictionary, stopwords_set)
    
    #Loading pre-processed files
    pi_file = open('positional_index_pickle_file', 'rb')
    positional_index = pickle.load(pi_file)
    pi_file.close()
    
    N = int(input())
    for q in range(N):
        input_sentence = input()
        sanitized_query = preprocess(input_sentence, stopwords_set, 'query')
        print(sanitized_query)
        query_size = len(sanitized_query)
        
        common_keys = [k for k in positional_index[sanitized_query[0]]]
        for query_term in sanitized_query[1:]:
            common_keys = get_common_keys(common_keys, positional_index[query_term])
        
        docs_satisfying_phrase_query = []
        for ck in common_keys:
            valid_indexes = positional_index[sanitized_query[0]][ck]
            for amount in range(1,query_size):
                next_indexes = [x-amount for x in positional_index[sanitized_query[amount]][ck]]
                valid_indexes = [x for x in valid_indexes if x in next_indexes]
            if len(valid_indexes)>0:
                docs_satisfying_phrase_query.append(ck)
        
        print('Number of documents matched: {}'.format(len(docs_satisfying_phrase_query)))
        print('Documents: \n')
        for doc_no in docs_satisfying_phrase_query:
            print(file_dictionary[doc_no])

In [28]:
if __name__ == "__main__":
    main()

1
american dream
['american', 'dream']
Number of documents matched: 3
Documents: 

Dataset/Humor,Hist,Media,Food/oliver.txt
Dataset/Humor,Hist,Media,Food/oliver02.txt
Dataset/Humor,Hist,Media,Food/p-law.hum
